In [1]:
!pip install -qq transformers datasets Bio

     |████████████████████████████████| 4.2 MB 4.3 MB/s 
     |████████████████████████████████| 346 kB 68.5 MB/s 
     |████████████████████████████████| 270 kB 68.1 MB/s 
     |████████████████████████████████| 596 kB 55.2 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 140 kB 59.8 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 
     |████████████████████████████████| 212 kB 46.6 MB/s 
     |████████████████████████████████| 127 kB 57.8 MB/s 
     |████████████████████████████████| 2.3 MB 32.9 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 74.0 MB/s 
     |████████████████████████████████| 144 kB 72.5 MB/s 
     |████████████████████████████████| 112 kB 70.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the p

In [1]:
from datasets import load_dataset
dna_test = load_dataset("simecek/Human_DNA_v0", split="test")

dna_test, len(dna_test[0]['Seq'])

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Using custom data configuration simecek--Human_DNA_v0-d7be3fc44fadbb72


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/simecek___parquet/simecek--Human_DNA_v0-d7be3fc44fadbb72/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


(Dataset({
     features: ['Seq'],
     num_rows: 29296
 }), 10000)

In [2]:
def kmers(s, k=6):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

kmers("ACTGACTA", 3)


"ACACACACACACAC"
"ACTGACAGATTAGA"

'ACTGACAGATTAGA'

In [5]:
from scipy.stats import entropy
import numpy as np

def entropy_kmers(s, k=2):
   value,counts = np.unique(kmers(s, k), return_counts=True)
   return entropy(counts)

entropy_kmers(dna_test[0]['Seq'])

2.683657518156041

In [6]:
triples = [(x['Seq'][0:3060], x['Seq'][3060:(2*3060)], x['Seq'][(2*3060):(3*3060)]) for x in dna_test]
chunks3060 = [item for sublist in triples for item in sublist]
len(chunks3060), len(chunks3060[0])

(87888, 3060)

In [7]:
import pandas as pd

chunks3060_df = pd.DataFrame({'Seq': chunks3060})
chunks3060_df

,Seq
0,TCACCAGCAATGTATGAAATTTCCAAGTGGTCTACATCCTTCTCTG...
1,TTGTGTTGGTAAATGAATGAATGAATGAGTCTGTTGGAAATTTTTA...
2,TTCTGACCTCTGTACTCAGTACCCAGCGCTTGTAGACCTGGATATG...
3,CAAACTTATGGGCATCATAGGGAGACCCTGTCTCAAAAAAAAAAAA...
4,AAAATTCATTTAGTTTTGCTTTTCATTTCACTTCTAATGTGTCTAA...
...,...
87883,CTTAAATCAAGTGCTGGGCTCACTTATTAGCTCCTGTCCTTAGATT...
87884,TTCATAAATTCTGCATACTAATCCATTGTGAGGAATAAGTGTCCTA...
87885,AAGAACCTAAAATGGCTCCGTACTGGCTATCACAACACTGTAAATT...
87886,CAGCATTCCTCTCCCAAAACAGATACTGGAAAGAAATATAATGTAT...


In [8]:
chunks3060_df['entropy'] = chunks3060_df.Seq.apply(entropy_kmers)

In [9]:
chunks3060_df.sort_values(['entropy'], inplace=True)
chunks3060_df

,Seq,entropy
52598,GATGGAGTGATGGAGTGATGCAGTGAGATGGAGTGATGGAGTGACG...,1.844400
4184,AGCCAGCCAAGCCAGCCAAGCCAGCCAAGCCAGTCAAGCCAGTCAG...,1.913351
4517,TCTGGTCTCTGGGTCCCTGTGGAGGCGGCTGGTCTCTGGGTCCCTG...,1.974530
4183,CAGCAAAGCCAGCCCAGCCAGCCAGCCAGCCAGCCAGCCGAGCCGG...,1.975730
20569,CATGCCAGCCAAGCCAGCCAGGGAGCCAAGCCAGCCAGCAAGCAAA...,1.998321
...,...,...
41970,CCAGTAAACATTCTTAAATACGTAAGCTGTGAGGTTATTCAGCAAA...,2.758866
46388,TAAACTTGCAAACTGGATACTAAGACACTTTCATGTCAGTTTTAAC...,2.759630
28865,TTTTAAGGTCTAAATGACCACTTCAAGCCATTCGTTTTAACAAAAG...,2.760014
51291,CAAAACCAGGTTTTTTGCTAATATGATCGAGGTGAAACATCAAGTT...,2.760141


In [10]:
from datasets import Dataset, DatasetDict

low_entropy = Dataset.from_pandas(chunks3060_df.head(30000))
medium_entropy = Dataset.from_pandas(chunks3060_df.head(57888).tail(27888))
high_entropy = Dataset.from_pandas(chunks3060_df.tail(30000))

ds = DatasetDict({'low': low_entropy, 'medium':medium_entropy, 'high': high_entropy})
ds

DatasetDict({
    low: Dataset({
        features: ['Seq', 'entropy', '__index_level_0__'],
        num_rows: 30000
    })
    medium: Dataset({
        features: ['Seq', 'entropy', '__index_level_0__'],
        num_rows: 27888
    })
    high: Dataset({
        features: ['Seq', 'entropy', '__index_level_0__'],
        num_rows: 30000
    })
})

## 3) Tokenization and chunking into 512 tokens tensors

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
def tokenize_function(s, k=6):
  seq_split = " ".join(kmers(s['Seq'], k))
  return tokenizer(seq_split)

tokenize_function({'Seq':'ACCTGCTGGACGATCATA'})  

{'input_ids': [2, 675, 2000, 393, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [14]:
tokenized_datasets = ds.map(tokenize_function, remove_columns=['Seq','entropy','__index_level_0__'] , num_proc=8)
tokenized_datasets

#0:   0%|          | 0/3750 [00:00<?, ?ex/s]

#1:   0%|          | 0/3750 [00:00<?, ?ex/s]

#2:   0%|          | 0/3750 [00:00<?, ?ex/s]

#3:   0%|          | 0/3750 [00:00<?, ?ex/s]

#4:   0%|          | 0/3750 [00:00<?, ?ex/s]

...

#4:   0%|          | 0/3750 [00:00<?, ?ex/s]

#5:   0%|          | 0/3750 [00:00<?, ?ex/s]

#6:   0%|          | 0/3750 [00:00<?, ?ex/s]

#7:   0%|          | 0/3750 [00:00<?, ?ex/s]

DatasetDict({
    low: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30000
    })
    medium: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27888
    })
    high: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30000
    })
})

In [15]:
len(tokenized_datasets['low'][0]['input_ids'])

512

In [16]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("simecek/DNADebertaSmall")

Downloading:   0%|          | 0.00/704 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/178M [00:00<?, ?B/s]

## 4) Evaluation

In [17]:
from transformers import TrainingArguments, Trainer

fake_training_args = TrainingArguments(
    output_dir='./model',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,   
    per_device_eval_batch_size=64,   
    max_steps=10,            # number of steps - to be the same
)

In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

In [19]:
results = []

for entropy_class in ['low', 'medium', 'high']:
  print(entropy_class)
  trainer = Trainer(
    model=model,
    args=fake_training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets[entropy_class],
    eval_dataset=tokenized_datasets[entropy_class])
  eval = trainer.evaluate()
  results.append((entropy_class, eval['eval_loss']))

low


max_steps is given, it will override any value given in num_train_epochs
***** Running Evaluation *****
  Num examples = 30000
  Batch size = 64


max_steps is given, it will override any value given in num_train_epochs
***** Running Evaluation *****
  Num examples = 27888
  Batch size = 64


medium


max_steps is given, it will override any value given in num_train_epochs
***** Running Evaluation *****
  Num examples = 30000
  Batch size = 64


high


In [22]:
import pandas as pd

results_long = pd.DataFrame.from_records(results, columns=["seqtype", "loss"])
results_long.round(2)

,seqtype,loss
0,low,6.01
1,medium,6.16
2,high,6.16


In [23]:
results_long.to_csv("loss_vs_entropy_class.csv")

In [ ]:
import torch 
torch.cuda.empty_cache() 